In [ ]:
#딥러닝 - CNN(컨볼루셔널 뉴럴 네크워크)

[이론]
다층 퍼셉트론 -> 단점  
1. 1차원으로 해석하기 때문에, 2차원 평면의 지역정보 잃음
2. 픽셀 하나의 변화에도 민감
3. 모델 크기 커야하고 학습 시간 오래걸리며 과대적합 가능성 높음
##CNN ->  
* 2차원 배열 그대로인 상태에서, 생김새 정보를 학습함  
* 필터(커널) :: (n*n행렬형태,대각선에 255,나머지에 0값)  
필터가 이동하며 생김새 정보 얻음  
이동기법 = 스트라이드  
필터와 겹치는 이미지 부분 = 수용영역  
* 필터가 이미지와 겹칠때 이미지 값과 필터 픽셀값을 곱한 합 -> 값이 크면 겹치는 부분 많다, 값이 작으면 겹치는 부분 적다


### 방식 :: 2*2필터, 5*5이미지 가정 -> 최종 행렬은 4*4형태로 나옴
* 풀링 레이어 :: 모델 파라미터를 줄임 -> 계산량 줄임, 과대적합 조절가능, 맥스풀링 -> 4*4형태를 2*2로 만들수 있음(2*2행렬의 가장 큰값을 가져옴)
* 제로 패딩 :: 0으로 이미지(입력행렬)을 감쌈 -> n+1*n+1 행렬이 됨 -> 필터를 3*3행렬로 늘리면, -> 전과같은 n+1*n+1행렬의 결과가 나옴
* 컬러 이미지 분류 :: ex) (10,28,28,3) 이면 이미지 개수10개, 28픽셀(가로세로), 색상 3가지라고 해석, 겹치는 레이어의 개수 = 깊이 (하나의CONV에 N개의 필터가 있다면, 깊이는 N이다)


###CNN 모델 학습 이해
1. 입력 레이어에 3가지(빨강,초록,파랑) 이미지가 들어가면 깊이는 3, (5,5,3), 5*5행렬
2. 첫번째 CONV1에는 4개의 필터 존재, 출력값으로 4개의 피처맵이 출력 (피처맵은 필터가 스트라이드를 거쳐 만든 결과 행렬), 4*4행렬 4개 (4,4,4)
3. CONV2에서는 총 3개의 필터 존재, 3개의 피처맵 출력, 4*4행렬 3개 (4,4,3)
4. 풀링 레이어에서 피처맵 크기 절반으로 줄임, 2*2행렬 3개 (2,2,3)
5. 모든 값을 일렬로 정렬, 2*2*3=12 배열 (12,1)
6. 이 12개 값은 CONV로 획득된 특징임 -> FC에 연결하여 0-9까지 값을 상징하는 10개의 노드로 값을 전달 -> softmax로 확률 출력 -> 옵티마이저로 역전파 이용해 파라미터 최적화

* 실습자는 필터의 개수와 형태만 결정 <-> 필터의 역할은 최적화 과정을 통해 모델 스스로 부여

#CNN[실습] :: 손글씨 학습 및 테스트
텐서플로로 MNIST 숫자 데이터 분류

In [ ]:
from IPython.display import Image
Image(url= "https://raw.githubusercontent.com/captainchargers/deeplearning/master/img/practice_cnn.png", width=800, height=200)


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# 항상 같은 결과를 갖기 위해 랜덤 시드 설정
tf.random.set_seed(1)

#라이브러리 임포트
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

MessageError: ignored

In [ ]:
#데이터 획득
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# 0-255까지 그레이스케일 확인
print(x_train[0][8])
print(y_train[0:9])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]
[5 0 4 1 9 2 1 3 1]


In [ ]:
#10000개의 샘플, 28*28이미지
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


In [ ]:
#데이터 구조 변경 -> 입력 레이어에 데이터 넣기 위함
import numpy as np
x_train = np.reshape(x_train, (60000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [ ]:
#데이터 정규화 -> 학습 시간 단축, 성능 높임
#값을 0,1사이값으로 정규화
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

In [ ]:
#실제값을 one hot encoding으로 변경 -> 손실함수에서 크로스 엔트로피 계산
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
#CNN 텐서플로우 구현
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=(28,28,1),padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True)]

# 코드 설명
# 첫번째 CONV에는 16개 필터, 크기 5*5, 편향값은 필터 개수만큼, 활성화 함수로 relu사용
# CONV레이어 다음으로 풀링 레이어로 액티베이션 맵 크기 줄임 -> 모델 크기 작아짐, 과대접합 위험도 감소
# 풀링 레이어로 맵크기는 14*14가 됨, 다음으로 CONV2에 입력, 32개의 필터, 풀링 레이어 이후 액티베이션 맵 크기 7*7
# FC는 CONV로 추출된 이미지 특징 입력받아 0-9숫자중 하나로 이미지 분류, flatten사용해 모든 특징들을 하나의 배열로 형변환, 형변환된 특징들은 FC에 입력
# FC영역에는 총 2개의 dense 레이어 존재, 첫번째 레이어는 128개 노드, 두번째 레이어는 10개 노드(10개의 노드값을 softmax가 입력해 각 노드별 확률 구하기위해), 각 노드는 0-9의미, 이 예측값은 크로스 엔트로피를 통해 실제값과의 차이 게산
# 크로스 엔트로피 + 아담 옵티마이저로 모델 최적화, 두번 이상 개선이 없으면 조기 종료

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1568)              0         
                                                                 
 dense (Dense)               (None, 128)               2

In [ ]:
#학습 시작&결과
model.fit(x_train, y_train,
          batch_size=500,
          epochs=5,
          verbose=1,
          validation_split = 0.1, 
          callbacks=callbacks)

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/5
108/108 [==============================] - 13s 13ms/step - loss: 0.4869 - accuracy: 0.8703 - val_loss: 0.1218 - val_accuracy: 0.9657
Epoch 2/5
108/108 [==============================] - 1s 11ms/step - loss: 0.1076 - accuracy: 0.9681 - val_loss: 0.0773 - val_accuracy: 0.9783
Epoch 3/5
108/108 [==============================] - 1s 11ms/step - loss: 0.0713 - accuracy: 0.9785 - val_loss: 0.0573 - val_accuracy: 0.9845
Epoch 4/5
108/108 [==============================] - 1s 10ms/step - loss: 0.0542 - accuracy: 0.9837 - val_loss: 0.0480 - val_accuracy: 0.9872
Epoch 5/5
108/108 [==============================] - 1s 10ms/step - loss: 0.0448 - accuracy: 0.9862 - val_loss: 0.0415 - val_accuracy: 0.9890
Test loss: 0.03843892365694046
Test accuracy: 0.9865000247955322
